In [1]:
from operator import add
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

In [2]:
# Download the files from AWS 

!wget -q  https://s3.amazonaws.com/metcs777/flights.csv.bz2
    
!wget -q  https://s3.amazonaws.com/metcs777/airports.csv.bz2

In [3]:
lines = sc.textFile("flights.csv.bz2")
airports_lines = sc.textFile('airports.csv.bz2')


In [4]:
# Removing the Header Line from CSV file
linesHeader = lines.first()
#header = sc.parallelize([linesHeader])
#linesWithOutHeader = lines.subtract(header)

flights_raw = lines\
    .zipWithIndex()\
    .filter(lambda x: x[1] > 1)\
    .keys()

#flishgts = flights_lines\
#    .map(lambda x: x.split(','))
#flights = flights_raw.map(lambda p: (p[0], p[1] , p[2] , p[3], p[4] , p[5] , p[6], p[7] , p[8] , p[9], p[10], '0' if p[11] == '' else p[11], p[24] ))


flights = flights_raw\
    .map(lambda x: x.split(','))\
    .map(lambda x: (x[0], x[1], x[2], x[3], x[4], 
                    x[5], x[6], x[7], x[8], x[9], 
                    x[10], 0 if x[11]=='' else float(x[11]), x[12], x[13], x[14], 
                    x[15], x[16], x[17], x[18], x[19], 
                    x[20], x[21], x[22], x[23], x[24],
                    x[25], x[26], x[27], x[28], x[29], x[30]
))


airports_raw = airports_lines\
    .zipWithIndex()\
    .filter(lambda x:x[1]>1)\
    .keys()

airports = airports_raw\
    .map(lambda x: x.split(','))\
    .map(lambda x: (x[0], x[1], x[2], x[3]))\
    .filter(lambda x: x[0])


mainFlightsData = flights.map(lambda p: (p[0], p[1], p[2], p[3], p[4], p[5],
                                         p[6], p[7], p[8], p[9], p[10], p[11], p[24]))

mainFlightsData.take(1)

[('2015',
  '1',
  '1',
  '4',
  'US',
  '840',
  'N171US',
  'SFO',
  'CLT',
  '0020',
  '0018',
  -2.0,
  '0')]

In [5]:
# number 6 is ORIGIN_AIRPORT
flightsFromBoston = mainFlightsData\
    .filter(lambda p: True if p[7] == "BOS" else False )

# Get the total number of Flights from BOS
# 107847 flights from Logan Airport in Boston
print(flightsFromBoston.count())

# Find the subset of flights departing on the weekend.
weekEndFlights = flightsFromBoston\
    .filter(lambda p: True if (int(p[3])==6 or int(p[3]) ==7) else False)
print(weekEndFlights.count()) # 26092

107847
26092


In [7]:
#Q1 Find a list of Origin Airports
result = flights\
    .map(lambda x:x[7])\
    .distinct()


# Save the RDD into a Folder
# This can be a path on Cloud Storage

# Just show one of the results 
result.take(1)

['SLC']

In [8]:
#Q2 Find a list of (Origin, Destination) pairs
result = flights\
    .map(lambda x: (x[7],x[8]))\
    .distinct()

# Save the RDD into a Folder
# This can be a path on Cloud Storage
# result.saveAsTextFile("Origin-Destination")

# Just show one of the results 
result.take(1)

[('SEA', 'ANC')]

In [9]:
#Q3 Find the Origin airport which had the largest departure delay in the month of January
result = flights\
    .filter(lambda x: x[1]=="1")\
    .map(lambda x: (x[7], x[11]) )\
    .top(1, lambda x:x[1])

print(result)

[('BHM', 1988.0)]


In [10]:
#Q4 Find out which carrier has the largest delay on Weekends. 
result = flights\
    .filter(lambda x: x[3]=="6" or x[3]=="7")\
    .map(lambda x: (x[4],x[11]) )\
    .top(1,lambda x: x[1])

print(result)

[('AA', 1878.0)]


In [11]:
#Q5 Which airport has the most cancellation of flights?
result = flights\
    .filter(lambda x: x[24]=="1")\
    .map(lambda x: (x[7],1))\
    .reduceByKey(add)\
    .top(1, lambda x: x[1])

print(result)

[('ORD', 8548)]


In [12]:
#Q6 Find the percent of flights cancelled for each carrier.
result = flights\
    .map(lambda x: (x[4], ((int(x[24])), 1)))\
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
    .map(lambda x: (x[0], x[1][0]/x[1][1] * 100))

# Just show one of the results 
result.take(1)


[('AA', 1.5040297086846388)]

In [13]:
#Q7 Find the largest departure delay for each carrier
result = flights\
    .map(lambda x: (x[4], x[11]) )\
    .reduceByKey(max)

# Just show one of the results 
result.take(1)

[('AA', 1988.0)]

In [14]:
#Q8 Find the largest departure delay for each carrier for each month
result = flights\
    .map(lambda x: ((x[4], int(x[1])), x[11]) )\
    .reduceByKey(max)

# Just show one of the results 
result.take(1)


[(('UA', 1), 886.0)]

In [15]:
#Q9 For each carrier find the average Departure delay
result = flights \
    .map(lambda x: (x[4], (x[11], 1))) \
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
    .map(lambda x: (x[0], x[1][0]/x[1][1])) \

# Just show one of the results 
result.take(1)


[('AA', 8.773542906652084)]

In [16]:
#Q10 For each carrier find the average Departure delay for each month
result = flights.map(lambda x: ((x[4], int(x[1])),(x[11],1)))\
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
    .map(lambda x: (x[0], x[1][0]/x[1][1]))

# Just show one of the results 
result.take(1)


[(('UA', 1), 13.666987889048054)]

In [17]:
#Q11 Which date of year has the highest rate  of flight cancellations?
# Rate of flight cancellation is calculated by deviding number of canceled flights by total number of flights.
result = flights\
    .map(lambda x: ((x[0],x[1],x[2]), ((int(x[24])), 1)) )\
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
    .map(lambda x: (x[0], x[1][0]/x[1][1]))\
    .top(1, lambda x:x[1])

print(result)


[(('2015', '1', '27'), 0.19030023094688223)]


In [18]:
#Q12 Calculate the number of flights to each destination state
# For each carrier, for which state do they have the largest average delay? 
# You will need the airline and airport data sets for this question.

result = flights\
    .map(lambda x: ( (x[4],x[8]), (x[11],1) ))\
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
    .map(lambda x: (x[0][1], (x[0][0], x[1][0]/x[1][1]) ))\
    .leftOuterJoin(airports.map(lambda x: (x[0], x[3])))\
    .filter(lambda x: x[1][1] is not None)\
    .map(lambda x: (x[1][0][0], (x[1][1], x[1][0][1])))\
    .reduceByKey(lambda x,y: (x[0],x[1]) if x[1] > y[1] else (y[0],y[1]) )\
    .map(lambda x: (x[0], x[1][0]))

# Just show one of the results 
result.take(1)


[('NK', 'PA')]